In [356]:
import pandas as pd

In [357]:
# Function with for loop to download all stats of the team internal - Dict as I called teams differently to the website
teams_internal_map = {'LAL':'LAL','CHI':'CHB','GSW':'GSW','MIA':'MIH','BOS':'BOC', 'BRK':'BRK', 'PHI':'PHI', 'MIL':'MIL', 'NYK':'NYK', 'ATL':'ATL', 'CHO':'CHO', 'IND':'IND', 'WAS':'WAS' , 'TOR':'TOR', 'CLE':'CLE', 'ORL':'ORL', 'DET':'DET', 'UTA':'UTA', 'PHO':'PHO', 'LAC':'LAC', 'DEN':'DEN', 'DAL':'DAL', 'POR':'POR', 'MEM':'MEM', 'SAS':'SAS', 'NOP':'NOP', 'SAC':'SAC', 'OKC':'OKC', 'MIN':'MIN', 'HOU':'HOU'}


In [358]:
# only for relevant teams
teams_internal = {'LAL':'LAL','CHI':'CHB','GSW':'GSW','MIA':'MIH','BOS':'BOC'}


In [359]:
# add stats of opponents and take difference between home and away team

for teams in teams_internal:

    # Add first line of the stats
    i = 0

    df = pd.read_excel('C:/Users/Marc/Dropbox/06_ESCP/01_Uni/06_MA Thesis/04_Code/02_Output/01_NBA Scheudle & Stats/'+ teams_internal[teams] + '/02_stats_average/'+ teams_internal[teams] + '_2021_stats_num_avg.xlsx')

    # No loop to create the df
    # open opponents dataset
    df_opp = pd.read_excel('C:/Users/Marc/Dropbox/06_ESCP/01_Uni/06_MA Thesis/04_Code/02_Output/01_NBA Scheudle & Stats/'+ teams_internal_map[df.iloc[i].Opp] + '/02_stats_average/'+ teams_internal_map[df.iloc[i].Opp] + '_2021_stats_num_avg.xlsx')

    # subset to numerical values in df
    df_opp_interesting = df_opp.iloc[:, 6:]

    # Add prefix to columns indicating that its from the opponent
    df_opp_interesting = df_opp_interesting.add_prefix('Opp_')

    # identify column to to add
    # check date of game
    condition = df.Date.iloc[i]

    # search index of matching date in df_opp as its not in df_opp_interesting; rows however the same
    x = df_opp[df_opp['Date'].str.match(condition)].index

    # take index and look for it in df_opp_interesting
    new = df_opp_interesting.iloc[x]

    ### repetition as now new lines will be added

    for i in range(1, len(df)):

        df = pd.read_excel('C:/Users/Marc/Dropbox/06_ESCP/01_Uni/06_MA Thesis/04_Code/02_Output/01_NBA Scheudle & Stats/'+ teams_internal[teams] + '/02_stats_average/'+ teams_internal[teams] + '_2021_stats_num_avg.xlsx')

        # No loop to create the df
        # open opponents dataset
        df_opp = pd.read_excel('C:/Users/Marc/Dropbox/06_ESCP/01_Uni/06_MA Thesis/04_Code/02_Output/01_NBA Scheudle & Stats/'+ teams_internal_map[df.iloc[i].Opp] + '/02_stats_average/'+ teams_internal_map[df.iloc[i].Opp] + '_2021_stats_num_avg.xlsx')

        # subset to numerical values in df
        df_opp_interesting = df_opp.iloc[:, 6:]

        # Add prefix to columns indicating that its from the opponent
        df_opp_interesting = df_opp_interesting.add_prefix('Opp_')

        # identify column to to add
        # check date of game
        condition = df.Date.iloc[i]

        # search index of matching date in df_opp as its not in df_opp_interesting; rows however the same
        x = df_opp[df_opp['Date'].str.match(condition)].index

        # take index and look for it in df_opp_interesting
        to_add_new = df_opp_interesting.iloc[x]

        # merge with home team dataset
        new = new.append(to_add_new)

    # reset index as sometime it is messed up (3rd and 4th LAL game is 3rd game for POR and MIL)
    new = new.reset_index()

    # Merge df with the opponents results (to_add)
    df_final = pd.concat([df, new], axis = 1)

    # Calc differences betweent home and opponent team
    #  In order to keep dimensionality at a minimum, the difference of all statistical values of H/A team are taken
    Diff_list = ['P_Team', 'P_Received', 'FG', 'FGA', '3P', '3PA', 'FT',
           'FTA', 'ORB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'FG%', '3P%',
           'FT%', 'P_Team_5GA', 'P_Rec_5GA', 'FG_5GA', 'FGA_5GA', '3P_5GA',
           '3PA_5GA', 'FT_5GA', 'FTA_5GA', 'ORB_5GA', 'TRB_5GA', 'AST_5GA',
           'STL_5GA', 'BLK_5GA', 'TOV_5GA', 'PF_5GA', 'FG%_5GA', '3P%_5GA',
           'FT%_5GA', 'win_per', 'win_per_5G', 'days_rest']

    for i in Diff_list:
        df_final['Diff_'+ i] = df[i] - df_final['Opp_' +i]
        df_final = df_final.drop(columns = [i,'Opp_' +i ])

    # Dropping index from concatination
    df_final = df_final.drop(columns = ['index'])

    # saving files to the repective paths
    save = df_final

    save.to_excel('C:/Users/Marc/Dropbox/06_ESCP/01_Uni/06_MA Thesis/04_Code/02_Output/01_NBA Scheudle & Stats/'+ str(teams_internal[teams]) + '/03_stats_merged with opponent/'+ str(teams_internal[teams]) + '_hist_data_overview.xlsx', index = False)


Main Problem:
- Matching the correct game from the opponent to the dataset, as it 5th game is not always the same
- Done by:
    - Lookup date of game
    - Search this date in opponent dataframe
    - return index of this row
    - Append row of that index

